In [28]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

dev_set_df=pd.read_csv("../data/assignment1_dev_set.csv")
val_set_df=pd.read_csv("../data/assignment1_val_set.csv")

print(list(dev_set_df))

bmi=dev_set_df["BMI"]
age=dev_set_df["Host age"]
sex=dev_set_df["Sex"]

to_drop=['Unnamed: 0', 'Project ID','Experiment type','Sex','Host age','Disease MESH ID','BMI']

bacteria=dev_set_df.drop(labels=to_drop,axis=1)

print(list(bacteria))
print(bmi)

for bacterium in list(bacteria):
    print(f"--{bacterium}--\n Mean: {bacteria[bacterium].mean()}\n Standard deviation: {bacteria[bacterium].std()}\n \
Max:{bacteria[bacterium].max()}\n Min:{bacteria[bacterium].min()}")


['Unnamed: 0', 'Project ID', 'Experiment type', 'Sex', 'Host age', 'BMI', 'Disease MESH ID', 'Acholeplasma axanthum', 'Acidaminococcus fermentans', 'Acidaminococcus intestini', 'Actinomyces lingnae', 'Akkermansia muciniphila', 'Alistipes finegoldii', 'Alistipes indistinctus', 'Alistipes obesi', 'Alistipes onderdonkii', 'Alistipes putredinis', 'Alistipes shahii', 'Alistipes sp. RMA 9912', 'Alistipes timonensis', 'Anaerofilum pentosovorans', 'Anaerofustis stercorihominis', 'Anaerostipes butyraticus', 'Anaerostipes caccae', 'Anaerotruncus colihominis', 'Bacillus megaterium', 'Bacillus nealsonii', 'Bacteroides eggerthii', 'Bacteroides fragilis', 'Bacteroides gallinarum', 'Bacteroides graminisolvens', 'Bacteroides intestinalis', 'Bacteroides massiliensis', 'Bacteroides ovatus', 'Bacteroides plebeius', 'Bacteroides salyersiae', 'Barnesiella viscericola', 'Bifidobacterium bifidum', 'Bifidobacterium pseudocatenulatum', 'Bifidobacterium saguini', 'Bifidobacterium tsurumiense', 'Bilophila wadswo